In [1]:
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
%load_ext sql

In [2]:
%sql sqlite:///results.db

# Scripts

Any alias that is a composition of two or more commands.

In [4]:
%%sql
with total as (select count(*) as total from alias)
select 
    count(distinct alias_id) as '#', 
    round(count(distinct alias_id)*100.0/total,2) as '%' 
from alias join command using (alias_id)
join total
where alias.num_commands != 1;


 * sqlite:///results.db
Done.


#,%
204142,9.26
